In [1]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz

import torch

from cell.utils import link_prediction_performance
from cell.cell import Cell, EdgeOverlapCriterion, LinkPredictionCriterion
from cell.graph_statistics import compute_graph_statistics

# Load graph and validation-/ test edges (same split as in paper)

In [2]:
train_graph = load_npz('./data/CORA-ML_train.npz')
with open('./data/link_prediction.p', 'rb') as handle:
    val_ones, val_zeros, test_ones, test_zeros = pickle.load(handle)

## Edge overlap criterion

In [3]:
# initialize model with EO-criterion
model = Cell(A=train_graph,
             H=9,
             callbacks=[EdgeOverlapCriterion(invoke_every=10, edge_overlap_limit=.5)])

In [4]:
# train model 
model.train(steps=200,
            optimizer_fn=torch.optim.Adam,
            optimizer_args={'lr': 0.1,
                            'weight_decay': 1e-7})

Step:  10/200 Loss: 6.01700 Edge-Overlap: 0.034 Total-Time: 4
Step:  20/200 Loss: 4.09000 Edge-Overlap: 0.280 Total-Time: 7
Step:  30/200 Loss: 3.32705 Edge-Overlap: 0.447 Total-Time: 11
Step:  40/200 Loss: 2.96411 Edge-Overlap: 0.556 Total-Time: 15


In [5]:
generated_graph = model.sample_graph()

In [6]:
compute_graph_statistics(generated_graph)

{'d_max': 204.0,
 'd_min': 1.0,
 'd': 4.8277580071174375,
 'LCC': 2804,
 'wedge_count': 81334.0,
 'claw_count': 1884377.0,
 'triangle_count': 1358,
 'square_count': 6851.0,
 'power_law_exp': 1.816566731479453,
 'gini': 0.44549792945835387,
 'rel_edge_distr_entropy': 0.9513050284843959,
 'assortativity': -0.06598683352654197,
 'clustering_coefficient': 0.050089753362677354,
 'cpl': 5.283193407109608}

In [7]:
compute_graph_statistics(train_graph)

{'d_max': 238.0,
 'd_min': 1.0,
 'd': 4.8277580071174375,
 'LCC': 2810,
 'wedge_count': 101747.0,
 'claw_count': 3033514.0,
 'triangle_count': 2802,
 'square_count': 14268.0,
 'power_law_exp': 1.8550648593086239,
 'gini': 0.4825742921255409,
 'rel_edge_distr_entropy': 0.9406652031225717,
 'assortativity': -0.07626405450439543,
 'clustering_coefficient': 0.08261668648707088,
 'cpl': 5.630006245811316}

## Validation criterion

In [8]:
# initialize model with LP-criterion
model = Cell(A=train_graph,
             H=9,
             callbacks=[LinkPredictionCriterion(invoke_every=2,
                                                val_ones=val_ones,
                                                val_zeros=val_zeros,
                                                max_patience=3)])

In [9]:
# train model 
model.train(steps=200,
            optimizer_fn=torch.optim.Adam,
            optimizer_args={'lr': 0.1,
                            'weight_decay': 1e-6})

Step:   2/200 Loss: 7.92396 ROC-AUC Score: 0.627 Average Precision: 0.631 Total-Time: 0
Step:   4/200 Loss: 7.70772 ROC-AUC Score: 0.710 Average Precision: 0.718 Total-Time: 1
Step:   6/200 Loss: 7.25824 ROC-AUC Score: 0.753 Average Precision: 0.768 Total-Time: 1
Step:   8/200 Loss: 6.61056 ROC-AUC Score: 0.789 Average Precision: 0.808 Total-Time: 2
Step:  10/200 Loss: 5.90689 ROC-AUC Score: 0.832 Average Precision: 0.852 Total-Time: 2
Step:  12/200 Loss: 5.30491 ROC-AUC Score: 0.871 Average Precision: 0.887 Total-Time: 3
Step:  14/200 Loss: 4.86908 ROC-AUC Score: 0.894 Average Precision: 0.907 Total-Time: 3
Step:  16/200 Loss: 4.55844 ROC-AUC Score: 0.903 Average Precision: 0.915 Total-Time: 4
Step:  18/200 Loss: 4.28209 ROC-AUC Score: 0.907 Average Precision: 0.917 Total-Time: 4
Step:  20/200 Loss: 4.04326 ROC-AUC Score: 0.912 Average Precision: 0.921 Total-Time: 5
Step:  22/200 Loss: 3.85592 ROC-AUC Score: 0.920 Average Precision: 0.930 Total-Time: 5
Step:  24/200 Loss: 3.68916 ROC-

#### Link prediction performance for ROC-AUC score and average precision

In [10]:
link_prediction_performance(scores_matrix=model._scores_matrix, val_ones=test_ones, val_zeros=test_zeros)

(0.94055625, 0.9441997059710999)